# 8. Hafta – ESA (3B CNN) ile Ses Sınıflandırma

**Not defteri adı:** `Week8_ESA_Audio_Kalp_Cevre.ipynb`  
**Klasör:** `C:\Users\The Coder Farmer\Desktop\AI LAB`

Bu dosya 3 bölümden oluşur:
- **Soru 1 (10p):** Kendi ESA (Conv3D) modelini tanımla.
- **Soru 2 (45p):** **Kalp** verisiyle eğitim — accuracy, F1, loss grafikleri + örnek tahmin.
- **Soru 3 (45p):** **Çevre** verisiyle eğitim — accuracy, F1, loss grafikleri + **confusion matrix**.

> Veri setini indirip köke veya sınıf klasörleriyle yerleştir. 



In [ ]:

# !pip install librosa soundfile tensorflow scikit-learn matplotlib numpy --quiet
import os, glob, re, random
import numpy as np
import librosa, soundfile as sf
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

SEED = 42
random.seed(SEED); np.random.seed(SEED); tf.random.set_seed(SEED)

BASE = r"C:\Users\The Coder Farmer\Desktop\AI LAB"

# GPU bellek büyümesini aç (varsa)
for g in tf.config.list_physical_devices('GPU'):
    try: tf.config.experimental.set_memory_growth(g, True)
    except: pass

ALLOW = {".wav",".mp3",".flac",".ogg",".m4a"}
SR, N_MELS, HOP, N_FFT = 22050, 64, 512, 1024
DEPTH, SLICE_W, BLOCK_HOP = 8, 8, 4

def label_from_name(fname):
    b = os.path.splitext(os.path.basename(fname))[0]
    m = re.match(r"^([A-Za-z]+)", b)
    return (m.group(1) if m else 'unknown').lower()

def list_audio_with_labels(root):
    sub = [d for d in os.listdir(root) if os.path.isdir(os.path.join(root,d))]
    items = []
    cls_dirs = [d for d in sub if any(os.path.splitext(p)[1].lower() in ALLOW for p in glob.glob(os.path.join(root,d,'*')))]
    if cls_dirs:   # class folders
        for c in cls_dirs:
            for p in glob.glob(os.path.join(root,c,'*')):
                if os.path.splitext(p)[1].lower() in ALLOW: items.append((p,c.lower()))
    else:          # flat folder
        for p in glob.glob(os.path.join(root,'*')):
            if os.path.splitext(p)[1].lower() in ALLOW: items.append((p,label_from_name(p)))
    return items

def melspec01(y, sr=SR):
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=N_FFT, hop_length=HOP, n_mels=N_MELS, power=2.0)
    S = librosa.power_to_db(S, ref=np.max)
    mn, mx = S.min(), S.max();  return (S - mn) / (mx - mn + 1e-8)

def spec_to_blocks(S01, depth=DEPTH, w=SLICE_W, hop=BLOCK_HOP):
    M, T = S01.shape; need = depth*w; out=[]
    for st in range(0, T-need+1, hop):
        stack = [S01[:, st+d*w: st+(d+1)*w] for d in range(depth)]
        out.append(np.stack(stack,0)[...,None].astype(np.float32))
    return np.asarray(out) if out else np.zeros((0, depth, M, w, 1), np.float32)

def load_dataset(root, max_blocks_per_file=30):
    pairs = list_audio_with_labels(root); assert pairs, f"Ses yok: {root}"
    labels = sorted({lab for _,lab in pairs}); lab2id = {l:i for i,l in enumerate(labels)}
    Xs, ys, files = [], [], []
    for p,lab in pairs:
        try:
            y,_ = librosa.load(p, sr=SR, mono=True)
            blks = spec_to_blocks(melspec01(y))
            if len(blks)==0: continue
            if max_blocks_per_file and len(blks)>max_blocks_per_file:
                idx = np.random.choice(len(blks), max_blocks_per_file, replace=False); blks = blks[idx]
            Xs.append(blks); ys.append(np.full((len(blks),), lab2id[lab], np.int64)); files += [p]*len(blks)
        except Exception as e:
            print("Skip:", p, e)
    return np.concatenate(Xs,0), np.concatenate(ys,0), labels, files

def onehot(y, n): oh=np.zeros((len(y),n),np.float32); oh[np.arange(len(y)),y]=1; return oh

def plot_curves(h, tag):
    fig, ax = plt.subplots(1,2, figsize=(10,4))
    ax[0].plot(h.history['accuracy']); ax[0].plot(h.history['val_accuracy']); ax[0].set_title(f'{tag} Accuracy'); ax[0].legend(['train','val']); ax[0].grid(True)
    ax[1].plot(h.history['loss']);     ax[1].plot(h.history['val_loss']);     ax[1].set_title(f'{tag} Loss');     ax[1].legend(['train','val']); ax[1].grid(True)
    plt.show()


## Soru 1 — Kendi ESA modelini oluştur (10p)

In [ ]:

def build_esa(input_shape, n_classes):
    inp = layers.Input(shape=input_shape)
    x = layers.Conv3D(16, 3, padding='same', activation='relu')(inp)
    x = layers.MaxPool3D((1,2,2))(x)
    x = layers.Conv3D(32, 3, padding='same', activation='relu')(x)
    x = layers.MaxPool3D((2,2,2))(x)
    x = layers.Conv3D(64, 3, padding='same', activation='relu')(x)
    x = layers.GlobalAveragePooling3D()(x)
    x = layers.Dense(128, activation='relu')(x)
    out = layers.Dense(n_classes, activation='softmax')(x)
    model = models.Model(inp, out, name='ESA_MyModel')
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                  loss='categorical_crossentropy', metrics=['accuracy'])
    return model

print("ESA modeli hazır.")


## Soru 2 — Kalp verisiyle ESA eğitimi (accuracy, F1, loss + tahmin)

In [ ]:

HEART_ROOT = os.path.join(BASE, "heart")  # Düz klasör veya class klasörleri
if not os.path.exists(HEART_ROOT): HEART_ROOT = BASE  # kökteyse dosya adına göre etiketlenecek

Xh, yh, ch, fh = load_dataset(HEART_ROOT, max_blocks_per_file=30)
idx = np.arange(len(yh))
Xtmp, Xt, ytmp, yt, itmp, it = train_test_split(Xh, yh, idx, test_size=0.2, stratify=yh, random_state=SEED)
Xtr, Xv, ytr, yv, itr, iv = train_test_split(Xtmp, ytmp, itmp, test_size=0.2, stratify=ytmp, random_state=SEED)

m = build_esa(Xtr.shape[1:], len(ch))
h = m.fit(Xtr, onehot(ytr,len(ch)), validation_data=(Xv, onehot(yv,len(ch))), epochs=20, batch_size=32, verbose=1)
plot_curves(h, "HEART")

yp = np.argmax(m.predict(Xt, verbose=0),1)
print(f"[HEART] Test Acc={accuracy_score(yt,yp):.4f} | F1(macro)={f1_score(yt,yp,average='macro'):.4f}")
print(classification_report(yt, yp, target_names=ch))

# Örnek tahminler
for j in np.random.choice(len(yt), size=min(6,len(yt)), replace=False):
    print(os.path.basename(fh[it]), "-> pred:", ch[yp[j]], "| true:", ch[yt[j]]); break


## Soru 3 — Çevre verisiyle ESA eğitimi (accuracy, F1, loss + confusion matrix)

In [ ]:

ENV_ROOT = os.path.join(BASE, "environment")
if not os.path.exists(ENV_ROOT): ENV_ROOT = BASE

Xe, ye, ce, fe = load_dataset(ENV_ROOT, max_blocks_per_file=30)
idx = np.arange(len(ye))
Xtmp, Xt, ytmp, yt, itmp, it = train_test_split(Xe, ye, idx, test_size=0.2, stratify=ye, random_state=SEED)
Xtr, Xv, ytr, yv, itr, iv = train_test_split(Xtmp, ytmp, itmp, test_size=0.2, stratify=ytmp, random_state=SEED)

m2 = build_esa(Xtr.shape[1:], len(ce))
h2 = m2.fit(Xtr, onehot(ytr,len(ce)), validation_data=(Xv, onehot(yv,len(ce))), epochs=20, batch_size=32, verbose=1)
plot_curves(h2, "ENV")

yp = np.argmax(m2.predict(Xt, verbose=0),1)
print(f"[ENV] Test Acc={accuracy_score(yt,yp):.4f} | F1(macro)={f1_score(yt,yp,average='macro'):.4f}")
print(classification_report(yt, yp, target_names=ce))

# Confusion matrix
cm = confusion_matrix(yt, yp, labels=range(len(ce)))
fig, ax = plt.subplots(figsize=(6,5))
im = ax.imshow(cm); ax.figure.colorbar(im, ax=ax)
ax.set(xticks=np.arange(len(ce)), yticks=np.arange(len(ce)),
       xticklabels=ce, yticklabels=ce, xlabel='Pred', ylabel='True', title='Environment – Confusion Matrix')
plt.setp(ax.get_xticklabels(), rotation=30, ha='right'); 
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        ax.text(j, i, cm[i,j], ha='center', va='center')
plt.tight_layout(); plt.show()
